In [106]:
import triton
import SupertracePybind as Supertrace
from supertrace_util import compatibleProcessing, initTritonCtxEnv, mergeRepeatIns, checkIndirectIns

In [107]:
# tracepath = "11111111-3.9.4.trace64"
tracepath = "deadbeef-3.9.4.trace64"

trace = Supertrace.parse_x64dbg_trace(tracepath)
record = trace.getRecord() # 获取trace的记录指令列表
print(f"trace instruction num: {len(record)}")

trace instruction num: 170728


In [108]:
modules = trace.user.meta.getModules()
for mod in modules:
    if mod.isMainModule:
        main_module = mod
        break

main_secs = main_module.getSections()
print("No\tName")
for i, sec in enumerate(main_secs):
    print(f"{i}\t{sec.name}")

No	Name
0	.text
1	.rdata
2	.data
3	.pdata
4	.u<l
5	.4R^
6	.gJ2
7	.rsrc
8	.reloc


In [109]:
vmpsec_begin = main_secs[4].addr
vmpsec_end = main_secs[6].addr
print(f"vmp begin: {hex(vmpsec_begin)}")
print(f"vmp end: {hex(vmpsec_end)}")

vmp begin: 0x140006000
vmp end: 0x1407da000


In [110]:
ctx = triton.TritonContext()
ctx.setArchitecture(triton.ARCH.X86_64)
ctx.setMode(triton.MODE.ALIGNED_MEMORY, True)
ctx.setMode(triton.MODE.AST_OPTIMIZATIONS, True)
ctx.setMode(triton.MODE.CONSTANT_FOLDING, True)
ctx.setMode(triton.MODE.ONLY_ON_TAINTED, True)
ctx.setMode(triton.MODE.TAINT_THROUGH_POINTERS, True) # 开启内存指针污染

In [111]:
record = mergeRepeatIns(ctx, record, True) # 合并trace里的rep指令
print(f"after mergeing 'rep' instructions, trace instruction num: {len(record)}")

after mergeing 'rep' instructions, trace instruction num: 170728


In [112]:
threads = trace.user.meta.getThreads()
for th in threads:
    if th.id == record[0].thread_id:
        main_thread = th
        break
print(f"main thread id: {main_thread.id} ({hex(main_thread.id)})")
print(f"teb: {hex(main_thread.teb)}") # 获取线程TEB地址
initTritonCtxEnv(ctx, record[0], main_thread.teb) # 初始化寄存器环境

main thread id: 3088 (0xc10)
teb: 0x47e7b21000


In [113]:
for i, ins in enumerate(record):
    if (i + 1 >= len(record)): nextIns = None
    else: nextIns = record[i + 1]

    ttins = triton.Instruction()
    ttins.setAddress(ins.ins_address)
    ttins.setOpcode(ins.bytes)
    ctx.disassembly(ttins)

    for memAcc in ins.mem_accs:
        if (memAcc.type == Supertrace.AccessType.READ and (vmpsec_begin <= memAcc.acc_address <= vmpsec_end)):
            ctx.taintMemory(triton.MemoryAccess(memAcc.acc_address, memAcc.acc_size)) # 污染来自VM区块的字节码（确保不会把混淆间接跳转也给识别进去）

    compatibleProcessing(ctx, ttins, ins, nextIns, True, False) # 执行指令

    if (checkIndirectIns(ttins) and (nextIns is not None) and ttins.isTainted()):
        jumpAddr = nextIns.ins_address
        print(f"{hex(ins.dbg_id)}\t{ttins.getDisassembly()}\t\t{hex(jumpAddr)}") # 打印指令序号、指令反汇编文本以及对应的跳转地址

0x96	ret		0x1406fb2d5
0xf9	ret		0x1406fb2d5
0x15c	ret		0x1406fb2d5
0x1bf	ret		0x1406fb2d5
0x222	ret		0x14075f8ad
0x288	jmp rbp		0x1406fb2d5
0x2eb	ret		0x1406fb2d5
0x34e	ret		0x1406fb2d5
0x3b1	ret		0x140495723
0x403	jmp rcx		0x1403af4ce
0x43d	jmp r11		0x14044be7c
0x47e	jmp rbp		0x140667e11
0x4b9	ret 0x10		0x1403ab876
0x51b	jmp rbp		0x1407b883f
0x55b	ret 0x10		0x1406e954b
0x5b4	ret 0x18		0x1406e954b
0x60d	ret 0x18		0x1406e954b
0x666	ret 0x18		0x1406e954b
0x6bf	ret 0x18		0x14042268f
0x716	jmp rbp		0x14036d4f3
0x77c	jmp rbp		0x14036d4f3
0x7e2	jmp rbp		0x1404f671a
0x816	ret 0x10		0x14069b96d
0x88c	jmp rbp		0x1404b3955
0x901	ret 8		0x140541546
0x951	jmp rbp		0x1406f179c
0x9a9	jmp rbx		0x1403b6e72
0x9fa	ret 8		0x1403b4c6a
0xa7e	jmp rbp		0x14069af6d
0xaca	jmp rbp		0x14032d73b
0xb27	jmp rbp		0x1406d361d
0xb6f	ret 0x18		0x140312106
0xba4	ret 8		0x140464b86
0xc25	jmp rbp		0x1406fc028
0xc62	jmp rdx		0x14067bb8c
0xcb6	ret 0x10		0x14032d73b
0xd13	jmp rbp		0x1406d361d
0xd5b	ret 0x18		0x140312106
0xd9